In [4]:
import numpy as np
import pandas as pd
import os
import sys

# Add the directory containing the VotingModel to the Python path
sys.path.append(os.path.abspath('/Users/idrees/Code/govxs/'))
from util.voting_rules import mean_aggregation, median_aggregation, quadratic_aggregation

from model.VotingModel import VotingModel

In [ ]:
# Parameters for the simulation
n_voters = 144
n_projects = 600
total_funds = 30e6
num_rounds = 100
voter_type='rn_model'
 # Can be "uniform", "rn", or "mallows"
dispersion = 0.1  # For Mallows model

# Initialize the model
model = VotingModel(voter_type=voter_type,num_voters=n_voters, num_projects=n_projects, total_op_tokens=total_funds)

# Define true values for the projects for the alignment metric
true_values = np.random.randint(0, 2, size=n_projects)  # Example: binary "true values"


def simulate_voting(model, num_rounds, metrics_functions, behavior_model, *args):
    results = []
    for _ in range(num_rounds):
        model.step(behavior_model, *args)
        round_results = {}
        for metric_name, metric_function in metrics_functions.items():
            round_results[metric_name] = metric_function(model)
        results.append(round_results)
    return pd.DataFrame(results)

# Define the metrics functions dictionary
metrics_functions = {
    "robustness": calculate_robustness,
    "social_welfare": calculate_social_welfare,
    "gini_index": calculate_gini_index,
    "group_strategyproofness": calculate_group_strategyproofness,
    "alignment_with_ground_truth": lambda m: calculate_alignment_with_ground_truth(m, true_values)
}

# Simulate the voting process
simulation_results = simulate_voting(model, num_rounds, metrics_functions, behavior_model, dispersion)

# Display the results
import ace_tools as tools; tools.display_dataframe_to_user(name="Simulation Results", dataframe=simulation_results)

# Analyze the results
def analyze_results(df):
    analysis = df.describe().transpose()
    return analysis

analysis_results = analyze_results(simulation_results)
tools.display_dataframe_to_user(name="Analysis Results", dataframe=analysis_results)


In [ ]:
n_voters_range = [50, 100, 200, 300, 400, 500, 600,700, 800, 900, 1000]
n_projects_range = [300,400,500 600,700,800,900, 1000]
total_funds_range = [1e6, 5e6, 10e6, 30e6,50e6]
dispersion_range = [0.01, 0.1, 0.5]
num_rounds = 100
behavior_model = "mallows"  # Example behavior model
def simulate_voting(n_voters, n_projects, total_funds, num_rounds, metrics_functions, behavior_model, *args):
    model = VotingModel(n_voters=n_voters, n_projects=n_projects, total_funds=total_funds)
    results = []
    for _ in range(num_rounds):
        model.step(behavior_model, *args)
        round_results = {}
        for metric_name, metric_function in metrics_functions.items():
            round_results[metric_name] = metric_function(model)
        results.append(round_results)
    return pd.DataFrame(results)
all_results = []

# Define true values for the projects for the alignment metric
true_values = np.random.randint(0, 2, size=max(n_projects_range))  # Ensure it covers the largest n_projects

# Define the metrics functions dictionary
metrics_functions = {
    "robustness": calculate_robustness,
    "social_welfare": calculate_social_welfare,
    "gini_index": calculate_gini_index,
    "group_strategyproofness": calculate_group_strategyproofness,
    "alignment_with_ground_truth": lambda m: calculate_alignment_with_ground_truth(m, true_values)
}

# Loop through each combination of parameters
for n_voters in n_voters_range:
    for n_projects in n_projects_range:
        for total_funds in total_funds_range:
            for dispersion in dispersion_range:
                print(f"Simulating for n_voters={n_voters}, n_projects={n_projects}, total_funds={total_funds}, dispersion={dispersion}")
                simulation_results = simulate_voting(
                    n_voters, n_projects, total_funds, num_rounds, metrics_functions, behavior_model, dispersion
                )
                simulation_results['n_voters'] = n_voters
                simulation_results['n_projects'] = n_projects
                simulation_results['total_funds'] = total_funds
                simulation_results['dispersion'] = dispersion
                all_results.append(simulation_results)

# Combine all results into a single DataFrame
all_results_df = pd.concat(all_results, ignore_index=True)

# Display the combined results
import ace_tools as tools; tools.display_dataframe_to_user(name="Parameter Sweep Results", dataframe=all_results_df)

# Analyze the results
def analyze_results(df):
    grouped = df.groupby(['n_voters', 'n_projects', 'total_funds', 'dispersion']).mean()
    return grouped

analysis_results = analyze_results(all_results_df)
tools.display_dataframe_to_user(name="Analysis Results", dataframe=analysis_results)
